In [1]:
import pandas as pd
from pathlib import Path
input_file = Path.cwd()/'receipts.csv'
df = pd.read_csv(input_file)

## Remove unwanted columns, add blank column and reorder

In [2]:
df.drop(['Given', 'Surname','Income','GST',], axis=1, inplace=True)

In [3]:
df["Cumulative"] = ""

In [4]:
df=df[['Number', 'Paid by', 'Date', 'Paid','Cumulative','Type', 'Payment_Type','Issued_By', 'Merchant Ref', 'Notes']]

In [5]:
df.head()

,Number,Paid by,Date,Paid,Cumulative,Type,Payment_Type,Issued_By,Merchant Ref,Notes
0,8013,Robert Oxford,2021-5-10 10:02:58,170.0,,Income,credit card,Rika Wylde,CM170458119_Oxford,"Enrolment into Drawing - Portraiture(AC35), st..."
1,8014,Lesley Puglisi,2021-5-10 11:07:43,20.0,,Income,credit card,Lesley Puglisi,CM930909418_Puglisi,Enrolment into Walking the Camino to Santiago ...
2,8015,Lorna Barnett,2021-5-10 11:36:26,15.0,,Income,credit card,Didi Mudigdo,CM153909196_Barnett,Membership Payment for (1-May-2021 to 11-Jul-2...
3,8015,Lorna Barnett,2021-5-10 11:36:26,150.0,,Income,credit card,Didi Mudigdo,CM153909196_Barnett,"Enrolment into Italian C1 Advanced(L26), start..."
4,8016,Frances Durrer,2021-5-10 12:42:31,15.0,,Saved,credit account,Didi Mudigdo,NaN,Membership adjustment saved to credit account


## Filter out non-credit card transactions
Now for the Boolean indexing. The rows we want to remove are 'Deposit'/'Saved' (from the Type column) and 'cash, 'EFT'/'cash' (from the Payment_Type) column. Constructing the Boolean index:

In [6]:
cash = df['Payment_Type']=='cash'
cheque = df['Payment_Type']=='cheque'
saved = df['Type']=='Saved'
eft = df['Payment_Type']=='EFT'
deposit = df['Type']=='Deposit'

And now apply and export:

In [7]:
df[~(cash|cheque|eft|saved|deposit)].to_csv('hurrah.csv',index=False)